In [ ]:
import shap
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
data = pd.read_csv('../input/divorce-prediction/divorce_data.csv', delimiter=';')
reference = pd.read_csv('../input/divorce-prediction/reference.tsv', delimiter='|')

In [ ]:
print(data.isnull().any().sum())
print(data.isna().any().sum())

In [ ]:
dist = sns.barplot(x=data['Divorce'].unique(), y=data['Divorce'].value_counts())
dist.set(xlabel = 'Divorce', ylabel = 'Count', title = 'Distribution of Divorce categories')

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(data.corr(), cmap="YlGnBu")
plt.show()

In [ ]:
train_x, test_x = train_test_split(data, test_size=0.3)
train_y = train_x.pop('Divorce')
test_y = test_x.pop('Divorce')

# RandomForestClassifier

In [ ]:
forest = RandomForestClassifier()
forest.fit(train_x, train_y)

preds = forest.predict_proba(test_x)[:,1]

In [ ]:
preds_bool = preds >= 0.5
accuracy = accuracy_score(test_y, preds_bool)
print('Accuracy :', accuracy)

In [ ]:
f1_value = f1_score(test_y, preds_bool)
print('F1 Score :', f1_value)

In [ ]:
print('Classification Report:\n',classification_report(test_y , preds_bool))

In [ ]:
explainer = shap.TreeExplainer(forest)
shap_value = explainer.shap_values(data)

In [ ]:
shap.summary_plot(shap_value, data, plot_type="bar", plot_size = (25,10))

# Use GridSearch

In [ ]:
param_grid={
    "max_depth" : np.arange(1, 20, 1),
    "min_samples_split": np.arange(2, 20, 2),
    "min_samples_leaf": np.arange(2, 20, 2)
}

In [ ]:
grid = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid,
    scoring="accuracy",
    cv=5
)
grid

In [ ]:
grid.fit(train_x, train_y)

In [ ]:
print(f'Best parameters {grid.best_params_}')

# DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(train_x, train_y)

preds_tree = tree.predict_proba(test_x)[:,1]

In [ ]:
tree_bool = preds_tree >= 0.5
accuracy = accuracy_score(test_y, tree_bool)
print('Accuracy :', accuracy)

In [ ]:
f1_value = f1_score(test_y, tree_bool)
print('F1 Score :', f1_value)

In [ ]:
print('Classification Report:\n',classification_report(test_y , tree_bool))